### NanoCube Benchmark - Pandas DataFrame vs. Cube on OLAP-style point queries
This benchmark compares the execution time of Pandas vs. NanoCube for a single semi-complex point query.
Use your own DataFrames and see what performance improvements you will gain.  

In [9]:
import random
import timeit
import string
from datetime import datetime
import pandas as pd
from nanocube import Cube

rows = 1_000_000
loops = 100

# Create a larger dataframe with 1M records
print(f"Creating Dataframe with {rows:,} rows and 5 columns ", end="")
start = datetime.now()
customers = string.ascii_uppercase
df = pd.DataFrame({'customer': random.choices(customers, weights=range(len(customers), 0, -1), k=rows),  # 'A', 'B', 'C', ...
                   'product':  random.choices([f'P{i}' for i in range(100)], weights=range(100, 0, -1), k=rows),  # 'P1', 'P2', 'P3', ...
                   'promo':    random.choices([True, False], k=rows),
                   'sales':    [int(random.random()*100) for _ in range(rows)],
                   'cost':     [int(random.random()*100) for _ in range(rows)]})
print(f"in {(datetime.now() - start).total_seconds():.5f} sec.")
print (df.head())

Creating Dataframe with 1,000,000 rows and 5 columns in 0.82991 sec.
  customer product  promo  sales  cost
0        K     P12   True     45    97
1        A     P21   True     63    45
2        M     P52   True     46     6
3        Q     P53  False     22    13
4        O     P37   True     44    59


In [10]:
# Create a cube
print(f"\nCreating and preparing Cube from Dataframe ", end="")
start = datetime.now()
cube = Cube(df)
print(f"in {(datetime.now() - start).total_seconds():.5f} sec.")



Creating and preparing Cube from Dataframe in 0.90824 sec.


In [11]:
# OLAP query using Pandas dataframe
q1 = 'df[(df["customer"] == "A") & (df["product"] == "P1")][["sales", "cost"]].sum()'
print(f"\nRunning OLAP-Queries with Pandas. Please wait...")
print(f"\tQuery 1: {q1}")
print(f"\tResult: {dict(df[(df['customer'] == 'A') & (df['product'] == 'P1')][['sales', 'cost']].sum().items())}")
q1_pd = timeit.timeit(q1, globals=globals(), number=loops)
print(f"\t{loops}x queries executed in {q1_pd:.5f} sec, avg. {q1_pd/loops:.5f} sec/query")


Running OLAP-Queries with Pandas. Please wait...
	Query 1: df[(df["customer"] == "A") & (df["product"] == "P1")][["sales", "cost"]].sum()
	Result: {'sales': 76596, 'cost': 76467}
	100x queries executed in 5.55870 sec, avg. 0.05559 sec/query


In [12]:

# OLAP query using Cube
q1 = 'cube.get(customer="A", product="P1")'
print(f"\nRunning OLAP-Queries with Cube. Don't wait...")
print(f"\tQuery 1: {q1}")
print(f"\tResult: {cube.get(customer='A', product='P1')}")
q1_cube = timeit.timeit(q1, globals=globals(), number=loops)
print(f"\t{loops}x queries executed in {q1_cube:.5f} sec, avg. {q1_cube/loops:.5f} sec/query")

print(f"\nBelieve it or not: Cube is {q1_pd/q1_cube:.0f}x times faster than Pandas DataFrame for OLAP queries.")


Running OLAP-Queries with Cube. Don't wait...
	Query 1: cube.get(customer="A", product="P1")
	Result: {'sales': 76596, 'cost': 76467}
	100x queries executed in 0.01477 sec, avg. 0.00015 sec/query

Believe it or not: Cube is 376x times faster than Pandas DataFrame for OLAP queries.
